In [2]:
from io import BytesIO
import base64
import matplotlib 
import pandas as pd
import pickle
import numpy as np
import scipy as sp
import math
import re
import pymysql as db
import os
#import cv2
import sklearn
import datetime
#from matplotlib import pyplot as plt
#from sshtunnel import SSHTunnelForwarder
from sklearn import model_selection, svm
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn import metrics
from sklearn.metrics import classification_report, average_precision_score, mean_squared_error
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import accuracy_score
#from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import label_binarize
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import r2_score
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import roc_curve, roc_auc_score

#from mlxtend.plotting import plot_decision_regions
import matplotlib.gridspec as gridspec
import itertools
import time

from pyglmnet import GLM
from catboost import CatBoostRegressor
import lightgbm as lgb
import xgboost as xgb
from xgboost import XGBClassifier
#from mlxtend.classifier import StackingClassifier

from scipy.io import loadmat
import scipy


#For Seaborn
import seaborn as sns
from IPython import get_ipython
get_ipython().run_line_magic('matplotlib', 'inline')
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 30,10
import matplotlib.pyplot as plt



#For Arima
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.arima_model import ARIMA

#For NLP
import nltk
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
import re
import string
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer,TfidfTransformer

#For Bayesian Optimization
#import bayes_opt
#from bayes_opt import BayesianOptimization


#Dask Framework
import dask.dataframe as dd

#For View
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

C:\Users\Nairomi\Anaconda\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
path='C:/Users/Nairomi/Documents/GitHub/Earthquake-Damage-Grade-Prediction/Data/'
mega_data=pd.read_csv(path+"mega_data.csv",sep=',',encoding="utf-8",error_bad_lines=False,low_memory = False)
print(mega_data.shape)

(631761, 54)


## Features on the Basis of District Level

##  has_ features 

In [6]:
has_features=['has_geotechnical_risk_fault_crack','has_geotechnical_risk_flood','has_geotechnical_risk_land_settlement',
       'has_geotechnical_risk_landslide', 'has_geotechnical_risk_liquefaction','has_geotechnical_risk_other', 
        'has_geotechnical_risk_rock_fall','has_repair_started','has_secondary_use', 'has_secondary_use_agriculture',
       'has_secondary_use_hotel', 'has_secondary_use_rental','has_secondary_use_institution', 'has_secondary_use_school',
       'has_secondary_use_industry', 'has_secondary_use_health_post','has_secondary_use_gov_office',
        'has_secondary_use_use_police','has_secondary_use_other','has_superstructure_adobe_mud',
       'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag','has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick','has_superstructure_cement_mortar_brick', 'has_superstructure_timber',
       'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered','has_superstructure_rc_engineered',
        'has_superstructure_other']
district_list=mega_data.district_id.unique().tolist()

### COUNT

In [7]:
#Creation has_features_count Columns
for i in has_features:
    mega_data[i+'_district_count']=0

In [ ]:
for i in has_features:
    for dist in district_list:
        temp_new = pd.DataFrame()
        temp_new = mega_data.loc[mega_data.district_id == dist, :]
        temp_new = temp_new.reset_index()
        temp_new.drop('index', axis=1, inplace=True)
        
        has_obj=temp_new[temp_new[i]==1]
        value=len(has_obj)
        
        mega_data[i+'_district_count']=np.where(mega_data['district_id']==dist, value, mega_data[i+'_district_count'])

### RATIO 
#### [No of Records Having 1] / [Total Number of Records]

In [46]:
#Creation has_features_ratio Columns
for i in has_features:
    mega_data[i+'_district_ratio']=0

In [47]:
for i in has_features:
    for dist in district_list:
        temp_new = pd.DataFrame()
        temp_new = mega_data.loc[mega_data.district_id == dist, :]
        temp_new = temp_new.reset_index()
        temp_new.drop('index', axis=1, inplace=True)
        
        has_obj=temp_new[temp_new[i]==1]
        value=len(has_obj)/len(temp_new)
        
        mega_data[i+'_district_ratio']=np.where(mega_data['district_id']==dist, value, mega_data[i+'_district_ratio'])

### AVERAGE RATIO
#### { [Number of Records Having 0] / [Total Number of Records] } / { [Number of Records Having 1] / [Total Number of Records] }

In [48]:
#Creation has_features_ratio Columns
for i in has_features:
    mega_data[i+'_district_average_ratio']=0

In [49]:
for i in has_features:
    for dist in district_list:
        temp_new = pd.DataFrame()
        temp_new = mega_data.loc[mega_data.district_id == dist, :]
        temp_new = temp_new.reset_index()
        temp_new.drop('index', axis=1, inplace=True)
        
        has_obj=temp_new[temp_new[i]==1]
        nos_obj=temp_new[temp_new[i]==0]
        value=(len(has_obj)/len(temp_new))/(len(nos_obj)/len(temp_new))
        
        mega_data[i+'_district_average_ratio']=np.where(mega_data['district_id']==dist, value, mega_data[i+'_district_average_ratio'])

## Features on the Basis of Municipal Level

In [50]:
has_features=['has_geotechnical_risk_fault_crack','has_geotechnical_risk_flood','has_geotechnical_risk_land_settlement',
       'has_geotechnical_risk_landslide', 'has_geotechnical_risk_liquefaction','has_geotechnical_risk_other', 
        'has_geotechnical_risk_rock_fall','has_repair_started','has_secondary_use', 'has_secondary_use_agriculture',
       'has_secondary_use_hotel', 'has_secondary_use_rental','has_secondary_use_institution', 'has_secondary_use_school',
       'has_secondary_use_industry', 'has_secondary_use_health_post','has_secondary_use_gov_office',
        'has_secondary_use_use_police','has_secondary_use_other','has_superstructure_adobe_mud',
       'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag','has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick','has_superstructure_cement_mortar_brick', 'has_superstructure_timber',
       'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered','has_superstructure_rc_engineered',
        'has_superstructure_other']
municipal_list=mega_data.vdcmun_id.unique().tolist()

### COUNT

In [51]:
#Creation has_features_count Columns
for i in has_features:
    mega_data[i+'_municipal_count']=0

In [52]:
for i in has_features:
    for mun in municipal_list:
        temp_new = pd.DataFrame()
        temp_new = mega_data.loc[mega_data.vdcmun_id == mun, :]
        temp_new = temp_new.reset_index()
        temp_new.drop('index', axis=1, inplace=True)
        
        has_obj=temp_new[temp_new[i]==1]
        value=len(has_obj)
        
        mega_data[i+'_municipal_count']=np.where(mega_data['vdcmun_id']==mun, value, mega_data[i+'_municipal_count'])

### RATIO 
#### [No of Records Having 1] / [Total Number of Records]

In [53]:
#Creation has_features_ratio Columns
for i in has_features:
    mega_data[i+'_municipal_ratio']=0

In [54]:
for i in has_features:
    for mun in municipal_list:
        temp_new = pd.DataFrame()
        temp_new = mega_data.loc[mega_data.vdcmun_id == mun, :]
        temp_new = temp_new.reset_index()
        temp_new.drop('index', axis=1, inplace=True)
        
        has_obj=temp_new[temp_new[i]==1]
        value=len(has_obj)/len(temp_new)
        
        mega_data[i+'_municipal_ratio']=np.where(mega_data['vdcmun_id']==mun, value, mega_data[i+'_municipal_ratio'])

### AVERAGE RATIO
#### { [Number of Records Having 0] / [Total Number of Records] } / { [Number of Records Having 1] / [Total Number of Records] }

In [55]:
#Creation has_features_ratio Columns
for i in has_features:
    mega_data[i+'_municipal_average_ratio']=0

In [56]:
for i in has_features:
    for mun in municipal_list:
        temp_new = pd.DataFrame()
        temp_new = mega_data.loc[mega_data.vdcmun_id == mun, :]
        temp_new = temp_new.reset_index()
        temp_new.drop('index', axis=1, inplace=True)
        
        has_obj=temp_new[temp_new[i]==1]
        nos_obj=temp_new[temp_new[i]==0]
        if(len(nos_obj)==0):
            value=0
        else:
            value=(len(has_obj)/len(temp_new))/(len(nos_obj)/len(temp_new))
        
        mega_data[i+'_municipal_average_ratio']=np.where(mega_data['vdcmun_id']==mun, value, mega_data[i+'_municipal_average_ratio'])

### AVERAGE NUMBER OF FLOORS BEFORE EARTHQUAKE  DISTRICT, MUNICIPAL and WARD


In [57]:
#List On District, Municipal and Ward Level
district_list=mega_data.district_id.unique().tolist()
municipal_list=mega_data.vdcmun_id.unique().tolist()

In [58]:
#District LEvel
mega_data['avg_floors_before_quake_district']=0
for dist in district_list:
        temp_new = pd.DataFrame()
        temp_new = mega_data.loc[mega_data.district_id == dist, :]
        temp_new = temp_new.reset_index()
        temp_new.drop('index', axis=1, inplace=True)
        
        number_of_floors=temp_new['count_floors_pre_eq'].sum()
        value=number_of_floors/len(temp_new)
        
        mega_data['avg_floors_before_quake_district']=np.where(mega_data['district_id']==dist, value, mega_data['avg_floors_before_quake_district'])

In [59]:
#Municipal LEvel
mega_data['avg_floors_before_quake_municipal']=0
for mun in municipal_list:
        temp_new = pd.DataFrame()
        temp_new = mega_data.loc[mega_data.vdcmun_id == mun, :]
        temp_new = temp_new.reset_index()
        temp_new.drop('index', axis=1, inplace=True)
        
        number_of_floors=temp_new['count_floors_pre_eq'].sum()
        value=number_of_floors/len(temp_new)
        
        mega_data['avg_floors_before_quake_municipal']=np.where(mega_data['vdcmun_id']==mun, value, mega_data['avg_floors_before_quake_municipal'])

### AVERAGE NUMBER OF FLOORS AFTER EARTHQUAKE  DISTRICT, MUNICIPAL and WARD

In [60]:
#District LEvel
mega_data['avg_floors_after_quake_district']=0
for dist in district_list:
        temp_new = pd.DataFrame()
        temp_new = mega_data.loc[mega_data.district_id == dist, :]
        temp_new = temp_new.reset_index()
        temp_new.drop('index', axis=1, inplace=True)
        
        number_of_floors=temp_new['count_floors_post_eq'].sum()
        value=number_of_floors/len(temp_new)
        
        mega_data['avg_floors_after_quake_district']=np.where(mega_data['district_id']==dist, value, mega_data['avg_floors_after_quake_district'])

In [61]:
#Municipal LEvel
mega_data['avg_floors_after_quake_municipal']=0
for mun in municipal_list:
        temp_new = pd.DataFrame()
        temp_new = mega_data.loc[mega_data.vdcmun_id == mun, :]
        temp_new = temp_new.reset_index()
        temp_new.drop('index', axis=1, inplace=True)
        
        number_of_floors=temp_new['count_floors_post_eq'].sum()
        value=number_of_floors/len(temp_new)
        
        mega_data['avg_floors_after_quake_municipal']=np.where(mega_data['vdcmun_id']==mun, value, mega_data['avg_floors_after_quake_municipal'])

### AVERAGE NUMBER OF FAMILIES IN A DISTRICT & MUNICIPAL

In [62]:
mega_data['avg_number_families_in_district']=0
mega_data['number_of_families_in_district']=0
for dist in district_list:
    temp_new=pd.DataFrame()
    temp_new=mega_data.loc[mega_data.district_id== dist,:]
    temp_new=temp_new.reset_index()
    temp_new.drop('index', axis=1, inplace=True)
    
    number_of_families=temp_new['count_families'].sum()
    value=number_of_families/len(temp_new)
    
    mega_data['avg_number_families_in_district']=np.where(mega_data['district_id']==dist, value, mega_data['avg_number_families_in_district'])
    mega_data['number_of_families_in_district']=np.where(mega_data['district_id']==dist, number_of_families, mega_data['number_of_families_in_district'])    

In [63]:
mega_data['avg_number_families_in_municipal']=0
mega_data['number_of_families_in_municipal']=0
for mun in municipal_list:
    temp_new=pd.DataFrame()
    temp_new=mega_data.loc[mega_data.vdcmun_id== mun,:]
    temp_new=temp_new.reset_index()
    temp_new.drop('index', axis=1, inplace=True)
    
    number_of_families=temp_new['count_families'].sum()
    value=number_of_families/len(temp_new)
    
    mega_data['avg_number_families_in_municipal']=np.where(mega_data['vdcmun_id']==mun, value, mega_data['avg_number_families_in_municipal'])
    mega_data['number_of_families_in_municipal']=np.where(mega_data['district_id']==dist, number_of_families, mega_data['number_of_families_in_municipal'])    